## Tools-Calling 机制

illufly 支持多种 Tools-Calling 标记。

- ToolCall: 实现 `<tool_call></tool_call>`，在 Ollama 风格的工具回调中常见此类格式
- SubTask: 实现 `<sub_task></sub_task>`，结构与 `<tool_call>` 类似，但处理逻辑有所不同
- Plans: 实现 `plan xxxx #E{n} function_name(arguments)`，输出多个工具回调时非常简洁

### 工具准备

In [1]:
from illufly.types import BaseAgent, ToolCall, SubTask, Plans

def get_city(location: str):
    """根据您的描述信息判断所在城市"""
    return "广州"

def get_weather(city: str):
    """获得天气情况"""
    return f"{city}是个晴天"

### ToolCall

In [22]:
text = """
**思考**

首先，需要确定广州的天气状况，以确认是否下雨。这将帮助我们决定是否需要进行羽毛球场的预订。因此，首要的任务是获取广州的天气信息。

**规划**

为了获得广州的天气信息，我将调用 `get_weather` 工具，并提供城市名称作为参数。

**行动**

<tool_call>
{
    "name": "get_weather",
    "arguments": {
        "city": "广州"
    }
}
</tool_call>
"""

tc = ToolCall([BaseAgent(get_weather)])
print(tc.extract_tools_call(text))

for block in tc.handle(text):
    if block.block_type == "chunk":
        print(block)

[{'function': {'name': 'get_weather', 'arguments': '{"city": "广州"}'}}]
广州是个晴天


### SubTask

In [17]:
text = """
**思考**

首先，需要确定广州的天气状况，以确认是否下雨。这将帮助我们决定是否需要进行羽毛球场的预订。因此，首要的任务是获取广州的天气信息。

**规划**

为了获得广州的天气信息，我将调用 `get_weather` 工具，并提供城市名称作为参数。

**行动**

<sub_task>
{
    "name": "get_weather",
    "arguments": {
        "city": "广州"
    }
}
</sub_task>
"""

st = SubTask([BaseAgent(get_weather)])
print(st.extract_tools_call(text))

for block in st.handle(text):
    if block.block_type == "chunk":
        print(block.text)

[{'function': {'name': 'get_weather', 'arguments': '{"city": "广州"}'}}]
广州是个晴天


### Plans

In [2]:
text = """
**思考**

首先，需要确定广州的天气状况，以确认是否下雨。这将帮助我们决定是否需要进行羽毛球场的预订。因此，首要的任务是获取广州的天气信息。

**规划**

为了获得广州的天气信息，我将调用 `get_weather` 工具，并提供城市名称作为参数。

**行动**

Plan: 获取城市位置. 
#E1 = get_city[{"location": "香江动物园"}]

Plan: 获取广州的天气情况. 
#E2 = get_weather[{"city": "#E1"}]
"""

p = Plans(tools_to_exec=[BaseAgent(t) for t in [get_weather, get_city]])
print(p.extract_tools_call(text))
for block in p.handle(text):
    if block.block_type == "chunk":
        print(block)

[{'id': '#E1', 'description': '获取城市位置.', 'name': 'get_city', 'arguments': '{"location": "香江动物园"}'}, {'id': '#E2', 'description': '获取广州的天气情况.', 'name': 'get_weather', 'arguments': '{"city": "#E1"}'}]
tool_to_exec {'function': {'name': 'get_city', 'arguments': '{"location": "香江动物园"}'}}
广州
tool_to_exec {'function': {'name': 'get_weather', 'arguments': '{"city": "广州"}'}}
广州是个晴天


[{'function': {'name': 'get_weather', 'arguments': '{"city": "广州"}'}}]

In [9]:
import re
pattern = r"Action:\s*(.*?)\nAction Input:\s*(\[.*?\])"
match = re.search(pattern, react_resp, re.DOTALL)
print(match.group(1).strip())
print(match.group(2).strip())


Multiplication Tool
[750, 12]
